<a href="https://colab.research.google.com/github/Awaish0419/Bond-Yield-Bootstrapper/blob/main/BondYieldBootstrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initial parameters

In [126]:
t_issue="2024-01-01"
t_maturity="2026-01-01"
face_value=100
coupon_rate=0.05
frequency="semi-annually"
market_price = 95

# Bond class definition

In [127]:
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from scipy.optimize import fsolve

class Bond:

  def generate_schedule(self):
    delta = {"monthly": 1, "quarterly": 3, "semi-annually": 6, "annually": 12}[self.frequency]
    period = 1
    date = self.issue_date + relativedelta(months=delta * period)
    while date <= self.maturity_date:
      self.payment_schedule.append(date)
      period += 1
      date = self.issue_date + relativedelta(months=delta * period)

  def __init__(self, issue_date, maturity_date, face_value, coupon_rate, frequency):
    self.issue_date = datetime.strptime(issue_date, "%Y-%m-%d")
    self.maturity_date = datetime.strptime(maturity_date, "%Y-%m-%d")
    self.face_value = face_value
    self.coupon_rate = coupon_rate
    self.frequency = frequency
    self.payment_schedule = []
    self.payments = []
    self.generate_schedule()

  def calculate_payments(self):
    self.payments = []
    coupon_payment = self.face_value * self.coupon_rate / {"monthly": 12, "quarterly": 4, "semi-annually": 2, "annually": 1}[self.frequency]
    for i, date in enumerate(self.payment_schedule):
      payment = coupon_payment
      if i == len(self.payment_schedule) - 1:
        payment += self.face_value
      self.payments.append((date.strftime("%Y-%m-%d"), payment))
    return pd.DataFrame(self.payments, columns=["Pay Date", "Pay Amount"])

  def npv(self, y):
    npv = 0
    for i, (_, payment) in enumerate(self.payments):
      ti = (i + 1) / {"monthly": 12, "quarterly": 4, "semi-annually": 2, "annually": 1}[self.frequency]
      npv += payment / (1 + y) ** ti
    return npv

  def calculate_yield(self, market_price):
    self.calculate_payments()
    def objective(y):
      return self.npv(y) - market_price

    yield_rate = fsolve(objective, 0.05)[0]
    return yield_rate

## Bond class test

In [128]:
bond = Bond(t_issue, t_maturity, face_value, coupon_rate, frequency)
df = bond.calculate_payments()
print(df.to_string(index=False))

yield_rate = bond.calculate_yield(market_price)
print(yield_rate)
print(f"Yield: {yield_rate:.4%}")

  Pay Date  Pay Amount
2024-07-01         2.5
2025-01-01         2.5
2025-07-01         2.5
2026-01-01       102.5
0.0789670946617533
Yield: 7.8967%


# Yield Curve class definition

In [129]:
import numpy as np
from scipy.interpolate import interp1d, CubicSpline
from datetime import datetime
import math

class YieldCurve:
  def __init__(self, maturities, yields, interpolation_method="linear", compounding_method="continuous"):
    self.maturities = np.array(maturities)
    self.yields = np.array(yields)
    self.compounding_method = compounding_method

    if interpolation_method == "linear":
      self.interpolator = interp1d(self.maturities, self.yields, kind="linear", fill_value="extrapolate")
    elif interpolation_method == "cubic":
      self.interpolator = CubicSpline(self.maturities, self.yields)
    else:
      raise ValueError("Unsupported interpolation method. Use 'linear' or 'cubic'.")

  def rate(self, date):
    return self.interpolator(date)

  def discount(self, date):
    y = self.rate(date)

    if self.compounding_method == "continuous":
      return math.exp(-y * date)
    elif self.compounding_method == "discrete":
      return 1 / ((1 + y) ** date)
    else:
      raise ValueError("Unsupported compounding method. Use 'continuous' or 'discrete'.")

## Yield Curve class test

In [130]:
maturities = [0.25, 0.5, 1.0, 5.0]
frequency = "monthly"
yields = []
for maturity in maturities:
  current_maturity = (datetime.strptime(t_issue, "%Y-%m-%d") + relativedelta(months=(int(12 * maturity)))).strftime("%Y-%m-%d")
  bond = Bond(t_issue, current_maturity, face_value, coupon_rate, frequency)
  yields.append(bond.calculate_yield(market_price))

print("maturities: ")
print(maturities)
print("yields: ")
print(yields)

yc = YieldCurve(maturities, yields, "cubic", "continuous")

print("Interpolated rate at 2 years:", yc.rate(2.0))
print("Discount factor at 2 years:", yc.discount(2.0))

maturities: 
[0.25, 0.5, 1.0, 5.0]
yields: 
[0.2916691632244055, 0.1659883561206477, 0.10781590684796283, 0.0634175736360265]
Interpolated rate at 2 years: 0.49245270930807217
Discount factor at 2 years: 0.3734745490059093


According to the result, we can see that the cubic interpolation method cannot make an accurate calculation. Firstly, the distances between maturities have a huge gap. (e.g. 5.0 - 1.0 >> 0.5 - 0.25) Secondly, the number of data is too small.

# Bootstrapper class

In [131]:
class Bootstrapper:
  def __init__(self, bond_data, interpolation_method="cubic", compounding_method="continuous"):
    self.bond_data = bond_data
    self.interpolation_method = interpolation_method
    self.compounding_method = compounding_method

  def bootstrap(self):
    maturities = []
    yields = []

    for bond in self.bond_data:
      maturity, price, frequency = bond
      coupon_rate = 0.05
      current_maturity = (datetime.strptime(t_issue, "%Y-%m-%d") + relativedelta(months=(int(12 * maturity)))).strftime("%Y-%m-%d")
      bond = Bond(t_issue, current_maturity, face_value, coupon_rate, frequency)

      yield_rate = bond.calculate_yield(price)
      maturities.append(maturity)
      yields.append(yield_rate)

    yc = YieldCurve(maturities, yields, self.interpolation_method, self.compounding_method)
    return yc

## Bootstrapper class test

In [132]:
from typing_extensions import final
bond_data = [(1, 97.2, 'annually'), (2, 98.5, 'semi-annually'), (3, 99.25, 'quarterly'), (4, 98.33, 'quarterly'), (5, 97.25, 'semi-annually')]
bootstrapper = Bootstrapper(bond_data)
final_yield_curve = bootstrapper.bootstrap()
df = pd.DataFrame({
  "Maturity": final_yield_curve.maturities,
  "yield": final_yield_curve.yields
})

print(df.to_string(index=False))

 Maturity    yield
        1 0.080247
        2 0.058895
        3 0.053771
        4 0.055808
        5 0.057183
